# Create context tiles

Create RGB tiles in Earth Engine that have encoded year, biomass loss, and intensity at different Z levels.

In [3]:
import folium
import ee
ee.Initialize()

In [8]:
def tile_url(image, viz_params=None):
    """Create a target url for WMS tiles of a given image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url

In [15]:
# UMD HANSEN DATA 2000 - 2016
umd = ee.Image('UMD/hansen/global_forest_change_2016_v1_4') # Load the Hansen et al. forest change dataset 
datamask = umd.select('datamask')
mask = datamask.eq(1)
land = mask # Make a land image out of the mask
land = land.mask(land) #  Mask land with itself to mask all the zeros (non-land)

# Woods Hole Research Center Biomass data
neotropic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Neotropic')
africa = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Africa')
australia = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Australia')
tropasia = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Tropical_Asia')
palearctic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Palearctic')
nearctic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Nearctic')

# A custom WHRC palette
whrcPALETTE = ("75322B,84512A,8E6232,da8c19,ef9e0b,ffc011,ffdb2d,"
"ffe215,f9eb46,d5e400,c9d800,becc00,b4c200,B7B95B,B2B659,AFB457,ABB156,"
"A6AE53,A3AB52,A1AA51,9FA950,9EA850,9CA74F,9BA64E,9AA54E,99A44D,95A24C,"
"92A04A,909E49,8C9C48,8B9A47,869745,859745,839544,839543,819443,7E9241,"
"7A8F40,778D3E,758C3E,758B3D,728A3C,71893C,70883B,6F873B,6D863A,6A8438,"
"678237,648036,627E37,607D34,5E7B33,5A7831,577630,53742E,50722D,4F712C,"
"4E702C,4C6F2B,4A6D2A,496D29,486C29,486C29,476B29,466A28,426827,3E6525,"
"3B6323,3A6223,396222,386122,355F21,345E22,315C1F,305B1E,2C591D,2B581C,"
"28561B,27551A,255419,245319,235218,225218,225118,215118,205017,1F4F17,"
"1C4E16,1B4D15,1A4C15,194C14,184A14,164913,154812,124711,114610,114610,"
"114610,114610")
                
myVis = {"max":400,"min":0, 'palette': whrcPALETTE}

# Make an image collection from the visualization images.
biomass = ee.ImageCollection([
  land.visualize({'palette': '75322B'}),
  africa.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
  australia.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
  nearctic.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
  neotropic.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
  palearctic.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
  tropasia.visualize({'palette': whrcPALETTE, "max":400, "min":0,}),
]).mosaic()

#Map.addLayer(biomass, null, "Biomass")

In [17]:
myVis

{'max': 400,
 'min': 0,
 'palette': '75322B,84512A,8E6232,da8c19,ef9e0b,ffc011,ffdb2d,ffe215,f9eb46,d5e400,c9d800,becc00,b4c200,B7B95B,B2B659,AFB457,ABB156,A6AE53,A3AB52,A1AA51,9FA950,9EA850,9CA74F,9BA64E,9AA54E,99A44D,95A24C,92A04A,909E49,8C9C48,8B9A47,869745,859745,839544,839543,819443,7E9241,7A8F40,778D3E,758C3E,758B3D,728A3C,71893C,70883B,6F873B,6D863A,6A8438,678237,648036,627E37,607D34,5E7B33,5A7831,577630,53742E,50722D,4F712C,4E702C,4C6F2B,4A6D2A,496D29,486C29,486C29,476B29,466A28,426827,3E6525,3B6323,3A6223,396222,386122,355F21,345E22,315C1F,305B1E,2C591D,2B581C,28561B,27551A,255419,245319,235218,225218,225118,215118,205017,1F4F17,1C4E16,1B4D15,1A4C15,194C14,184A14,164913,154812,124711,114610,114610,114610,114610'}

In [18]:
tile_url(biomass)

EEException: Image.visualize: Expected a string or list of strings for field 'bands'.

In [ ]:
basemap = 'Cartodb dark_matter'
tileset=r"https://storage.googleapis.com/landsat-cache/2016/{z}/{x}/{y}.png"
map = folium.Map(location=[28.29, -16.6], zoom_start=3, tiles=basemap)
map.add_tile_layer(tiles=tileset, max_zoom=12,min_zoom=1,  attr='Custom tiles')
map